Clean Version of COW


In [ ]:
#6/17
# stop using span. There are too many spans. Too much run time

# find the required sentences
# print out stuff between the <span> and </span> tags along with the line number where sentence is seen

#6/19
#leave context. Goal is to find companies that implement law.
#Return line number and pattern sentence enclosed within "."


#6/30
# Take all <span> enclosed material and store in a list
# Use search function on this list
# Get values




#7/10

# Mention 10-k format
# Header of section



In [ ]:
#COW Docs
# **DGCL ->       https://delcode.delaware.gov/title8/c001/
# Section 122(17) -> law that waives COW
# 10-k ->  ITEM 15.EXHIBITS, FINANCIAL STATEMENT SCHEDULES -> 3.1, 3.2 ->  Articles of Incorporation and By-Laws
# 

In [3]:
#Harvard Articles
# https://corpgov.law.harvard.edu/2019/03/11/corporate-opportunity-waivers-in-private-equity-and-venture-capital-investments/
# https://corpgov.law.harvard.edu/2020/02/27/corporate-opportunity-doctrine-litigation-continues-into-2020/

========================= PULLING THE DATA=============================                        

In [44]:
# Pulling CIK from companies.csv 
# Given by Professor for testing

# Store CIK into a list and feed into queryAPI through the CIK parameter

import pandas as pd
companies = pd.read_csv(r"companies.csv")
companies['cik'] = companies['cik'].astype('Int64')
companiesCIK = companies["cik"]
companiesCIKList = companiesCIK.tolist()
companiesCIKListTrimmed = companiesCIKList[:10]
companiesCIKListTrimmed
#lengthCIKList = len(companiesCIKList)
#print(lengthCIKList)

[1750, 61478, 730469, <NA>, 6201, 3197, 2601, 859163, 764622, 1808834]

In [45]:
companiesCIKListTrimmed = companiesCIKListTrimmed[:3] + companiesCIKListTrimmed[4:]
companiesCIKListTrimmed

[1750, 61478, 730469, 6201, 3197, 2601, 859163, 764622, 1808834]

pandas.core.series.Series

---------------------------LIBRARIES------------------------------------------------


In [2]:
#########################STEP 1

import requests
import time
import re
import pandas as pd
from sec_api import QueryApi
queryApi = QueryApi(api_key="fccd7cd584373afdfd88b43603c9154b6e32f09659ca642276c6cb8e8a2d01dc")

----------------------------API CALL------------------------------------------------------

In [46]:
queryApi = QueryApi(api_key="fccd7cd584373afdfd88b43603c9154b6e32f09659ca642276c6cb8e8a2d01dc")

query = {
  "query": { "query_string": { 
      "query": "formType:\"10-K\" AND entities.stateOfIncorporation: \"DE\" AND  filedAt:{2019-01-01 TO 2021-06-20}" 
    } },
  "from": "0",
  "size": "20",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings10kDE = queryApi.get_filings(query)

In [46]:
api_key = "fccd7cd584373afdfd88b43603c9154b6e32f09659ca642276c6cb8e8a2d01dc"
queryApi = QueryApi(api_key=api_key)
cikString = " OR ".join([f"{cik}" for cik in companiesCIKListTrimmed])
print(cikString)
# Build the query dynamically


1750 OR 61478 OR 730469 OR 6201 OR 3197 OR 2601 OR 859163 OR 764622 OR 1808834


'1750 OR 61478 OR 730469 OR <NA> OR 6201 OR 3197 OR 2601 OR 859163 OR 764622 OR 1808834'

In [47]:
query = {
  "query": { 
    "query_string": { 
      "query": f"cik:({cikString})"
  }},
  "from": "0",
  "size": "20",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

# Get filings
filingsCIKListTrimmed = queryApi.get_filings(query)

# Print the results



In [51]:
#print(filingsCIKListTrimmed)


{'total': {'value': 10000, 'relation': 'gte'}, 'query': {'from': 0, 'size': 20}, 'filings': [{'id': 'e2d846c3adc49908b6d2709d5b6cf45c', 'accessionNo': '0001104659-24-075153', 'cik': '1750', 'ticker': 'AIR', 'companyName': 'AAR CORP', 'companyNameLong': 'AAR CORP (Filer)', 'formType': '8-K', 'description': 'Form 8-K - Current report - Item 5.02', 'filedAt': '2024-06-26T16:05:35-04:00', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1750/000110465924075153/0001104659-24-075153.txt', 'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1750/000110465924075153/0001104659-24-075153-index.htm', 'linkToXbrl': '', 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1750/000110465924075153/tm2418233d1_8k.htm', 'entities': [{'companyName': 'AAR CORP (Filer)', 'cik': '1750', 'irsNo': '362334820', 'stateOfIncorporation': 'DE', 'fiscalYearEnd': '0531', 'type': '8-K', 'act': '34', 'fileNo': '001-06263', 'filmNo': '241073452', 'sic': '3720 Aircraft &amp; Parts', 'undefined': '04 

In [13]:
len(filings10kDE["filings"])

NameError: name 'filings10kDE' is not defined

In [51]:
#type(filings10kDE)
companiesParamListTrimmed = []
companyParamsTrimmed = {}

for eachCompany in range(len(filings10kDE["filings"])):
    cik = filings10kDE["filings"][eachCompany]["cik"]#["companyName"]
    companyName = filings10kDE["filings"][eachCompany]["companyName"]#["companyName"] 
    periodOfReport = filings10kDE["filings"][eachCompany]["periodOfReport"]#["companyName"]
    linkToTxt = filings10kDE["filings"][eachCompany]["linkToTxt"]
    companyParamsTrimmed[cik] = {
        "cik": cik,
        "companyName": companyName,
        "periodOfReport": periodOfReport,
        "linkToTxt": linkToTxt 
        
    }
    companiesParamListTrimmed.append(companyParams)#filings10kDE["filings"][0]["companyName"]#["companyName"]
#filings10kDE["filings"][0]["cik"]#["companyName"]
#filings10kDE["filings"][0]["periodOfReport"]#["companyName"]

#for x in filings10kDE:
#    print(filings10kDE[x])
companiesParamListTrimmed[0]

{'728447': {'cik': '728447',
  'companyName': 'EVO Transportation & Energy Services, Inc.',
  'periodOfReport': '2019-12-31',
  'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/728447/000156459021042927/0001564590-21-042927.txt'},
 '1100397': {'cik': '1100397',
  'companyName': 'Advaxis, Inc.',
  'periodOfReport': '2020-10-31',
  'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1100397/000149315221004926/0001493152-21-004926.txt'},
 '1816261': {'cik': '1816261',
  'companyName': 'Executive Network Partnering Corp',
  'periodOfReport': '2020-12-31',
  'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1816261/000119312521178612/0001193125-21-178612.txt'},
 '1869974': {'cik': '1869974',
  'companyName': 'Aesther Healthcare Acquisition Corp.',
  'periodOfReport': '2021-12-31',
  'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1869974/000149315222002707/0001493152-22-002707.txt'},
 '1334325': {'cik': '1334325',
  'companyName': 'Coda Octopus Group, Inc.',
  'periodOfRepo

In [48]:
print(filings10kDE)  #check


{'total': {'value': 10000, 'relation': 'gte'}, 'query': {'from': 0, 'size': 20}, 'filings': [{'ticker': 'TEUM', 'formType': '10-K', 'accessionNo': '0001628280-21-012457', 'cik': '1084384', 'companyNameLong': 'PARETEUM Corp (Filer)', 'effectivenessDate': '2020-12-31', 'companyName': 'PARETEUM Corp', 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1084384/000162828021012457/teum-20201231.htm', 'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1084384/000162828021012457/0001628280-21-012457.txt', 'filedAt': '2021-06-17T16:06:01-04:00', 'documentFormatFiles': [{'sequence': '1', 'size': '2947596', 'documentUrl': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/1084384/000162828021012457/teum-20201231.htm', 'description': '10-K', 'type': '10-K'}, {'sequence': '2', 'size': '285525', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1084384/000162828021012457/exhibit-31certificateofin

---------------------------PULLING URLS FROM DICTIONARY (TEST)------------------

In [52]:
#Code Works
# 20 URLs copied onto 10kDE.txt
# 20 URLs copied onto filingsCIKListTrimmed.txt


# Access the 'filings' key in the dictionary
#filings = filings10kDE.get('filings', [])
filings = filingsCIKListTrimmed.get('filings', [])

# Open a file in write mode to store the URLs
with open('filingsCIKListTrimmed.txt', 'w') as file:
    # Iterate over each entry in the 'filings' list
    for entry in filings:
        # Get the value of the 'linkToTxt' key
        link_to_txt = entry.get('linkToTxt', '')
        # Write the URL to the file
        file.write(link_to_txt + '\n')


------------------------------BASE QUERY------------------------------------------

In [1]:
#########################STEP 2




#Code works

#Step 1
#Base query

base_query = {
  "query": "PLACEHOLDER", # this will be set during runtime 
  "from": "0",
  "size": "200", # dont change this
  # sort by filedAt
  "sort": [{ "filedAt": { "order": "desc" } }]
}

-----------------------------UNIVERSAL QUERY-----------------------------------------------

In [3]:
##########################STEP 3


#Code works

#Step 2: 
#Universal Query

#pastes 10kDE URLS onto 10kDE_extended.txt
companyParams = {}
# open the file we use to store the filing URLs
log_file = open("10kDE_extended.txt", "a")


# start with filings filed in 2021, then 2020, 2019, ... up to 2010 
# uncomment line below to fetch all filings filed in 2022-2010
# for year in range(2021, 2009, -1):
for year in range(2022, 2020, -1):
    print("starting {year}".format(year=year))
    
    # a single search universe is represented as a month of the given year
    for month in range(1, 13, 1):
        # get 10-Q and 10-Q/A filings filed in year and month
        # resulting query example: "formType:\"10-Q\" AND filedAt:[2021-01-01 TO 2021-01-31]"
        universe_query = (
            "formType:\"10-K\" AND "
            "filedAt:[{year}-{month:02d}-01 TO {year}-{month:02d}-31] AND "
            "entities.stateOfIncorporation:\"DE\""
        ).format(year=year, month=month)
        
        print(universe_query)
        # set new query universe for year-month combination
        base_query["query"] = universe_query;

        # paginate through results by increasing "from" parameter 
        # until we don't find any matches anymore
        # uncomment line below to fetch 10,000 filings
        for from_batch in range(0, 9800, 200): 
        #for from_batch in range(0, 400, 200):
            # set new "from" starting position of search 
            base_query["from"] = from_batch;

            response = queryApi.get_filings(base_query)
            
            for eachCompany in range(len(response["filings"])):
                cik = response["filings"][eachCompany]["cik"]#["companyName"]
                companyName = response["filings"][eachCompany]["companyName"]#["companyName"] 
                periodOfReport = response["filings"][eachCompany]["periodOfReport"]#["companyName"]
                linkToTxt = response["filings"][eachCompany]["linkToTxt"]

                companyParams[cik] = {
                    "cik": cik,
                    "companyName": companyName,
                    "periodOfReport": periodOfReport,
                    "linkToTxt": linkToTxt 

                }
                #companiesParamList.append(companyParams)

            # no more filings in search universe
            if len(response["filings"]) == 0:
                break;

            # for each filing, only save the URL pointing to the filing itself 
            # and ignore all other data. 
            # the URL is set in the dict key "linkToFilingDetails"
            urls_list = list(map(lambda x: x["linkToFilingDetails"], response["filings"]))

            # transform list of URLs into one string by joining all list elements
            # and add a new-line character between each element.
            urls_string = "\n".join(urls_list) + "\n"
            
            log_file.write(urls_string)

log_file.close()


starting 2022
formType:"10-K" AND filedAt:[2022-01-01 TO 2022-01-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-02-01 TO 2022-02-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-03-01 TO 2022-03-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-04-01 TO 2022-04-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-05-01 TO 2022-05-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-06-01 TO 2022-06-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-07-01 TO 2022-07-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-08-01 TO 2022-08-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-09-01 TO 2022-09-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-10-01 TO 2022-10-31] AND entities.stateOfIncorporation:"DE"
formType:"10-K" AND filedAt:[2022-11-01 TO 202

In [11]:
#########################STEP 4

companyParams
#len(companyParams)

4359

In [12]:
#check
log_file = open("10kDE_extended.txt", "r")
print(log_file.read()[0:2000])
log_file.close()

https://www.sec.gov/Archives/edgar/data/728447/000095017022000602/evoa-20201231.htm
https://www.sec.gov/Archives/edgar/data/1100397/000149315222002766/formnt10-k.htm
https://www.sec.gov/Archives/edgar/data/1816261/000119312522023409/d221966d10ka.htm
https://www.sec.gov/Archives/edgar/data/1869974/000149315222002707/form10-k.htm
https://www.sec.gov/Archives/edgar/data/1334325/000149315222002705/formnt10-k.htm
https://www.sec.gov/Archives/edgar/data/781902/000160706222000053/ibal063021formnt10k.htm
https://www.sec.gov/Archives/edgar/data/12927/000001292722000010/ba-20211231.htm
https://www.sec.gov/Archives/edgar/data/1814215/000110465922009084/twnd-20211231x10ka.htm
https://www.sec.gov/Archives/edgar/data/1801762/000149315222002559/formnt10-k.htm
https://www.sec.gov/Archives/edgar/data/1260221/000126022122000004/tdg-20210930.htm
https://www.sec.gov/Archives/edgar/data/920760/000162828022001450/len-20211130.htm
https://www.sec.gov/Archives/edgar/data/1084580/000108458022000006/jef-2021113

-------------------------------READING URLS FROM .TXT FILE---------------------------------------------

In [ ]:
file_path = "10kDE_extended.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    DE10kLinks = file.read()
DE10kLinks

In [55]:
file_path = "filingsCIKListTrimmed.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    filingsCIKListTrimmedLinks = file.read()
print(filingsCIKListTrimmedLinks)

https://www.sec.gov/Archives/edgar/data/1750/000110465924075153/0001104659-24-075153.txt
https://www.sec.gov/Archives/edgar/data/1808834/000180883424000086/0001808834-24-000086.txt
https://www.sec.gov/Archives/edgar/data/764622/000076462224000060/0000764622-24-000060.txt
https://www.sec.gov/Archives/edgar/data/764622/000076462224000057/0000764622-24-000057.txt
https://www.sec.gov/Archives/edgar/data/764622/000110465924070030/0001104659-24-070030.txt
https://www.sec.gov/Archives/edgar/data/6201/000166427224000283/0001664272-24-000283.txt
https://www.sec.gov/Archives/edgar/data/6201/000166427224000282/0001664272-24-000282.txt
https://www.sec.gov/Archives/edgar/data/6201/000166427224000281/0001664272-24-000281.txt
https://www.sec.gov/Archives/edgar/data/6201/000166427224000280/0001664272-24-000280.txt
https://www.sec.gov/Archives/edgar/data/6201/000166427224000279/0001664272-24-000279.txt
https://www.sec.gov/Archives/edgar/data/6201/000166427224000278/0001664272-24-000278.txt
https://www.

---------------------------STORING CONTENTS OF URL ON LOCAL DEVICE --------------------------

In [ ]:
#Code works!
# Case 1: 10 file locally -- test
# Case 2: All files locally -- main

import re

cik_pattern = r'/data/(\d+)/'

headers = {
    'User-Agent': 'MyEDGARScript/1.0 (varghesb@bc.edu)'  # Replace with your email
}

# Case 1
#for i in range(10):
for url in DE10kLinks: 
   
    #url = DE10kLinks[i]                  # Case 1
    
    match = re.search(cik_pattern, url)
    if match:
        cik = match.group(1)
    else:
        cik = 'unknown'
    # Make the request with the headers
    r = requests.get(url, headers=headers)
    
    # Define the file path for each file
    file_path = f"C:\\Users\\sidha\\OneDrive\\Sid\\MSF\\Summer\\RA\\Code\\10-K\\DE\\{cik}.txt"
    # Save the response text to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(r.text)
    
    print(f"Response saved to {file_path}")
    
    # Wait for a few seconds before making another request (adjust the number of seconds as per the rate limit)
    time.sleep(0.1)

In [72]:
#filingsCIKListTrimmedLinks.split("\n")
#filingsCIKListTrimmedLinksList = [] 


filingsCIKListTrimmedLinksList = filingsCIKListTrimmedLinks.split()
filingsCIKListTrimmedLinksList

['https://www.sec.gov/Archives/edgar/data/1750/000110465924075153/0001104659-24-075153.txt',
 'https://www.sec.gov/Archives/edgar/data/1808834/000180883424000086/0001808834-24-000086.txt',
 'https://www.sec.gov/Archives/edgar/data/764622/000076462224000060/0000764622-24-000060.txt',
 'https://www.sec.gov/Archives/edgar/data/764622/000076462224000057/0000764622-24-000057.txt',
 'https://www.sec.gov/Archives/edgar/data/764622/000110465924070030/0001104659-24-070030.txt',
 'https://www.sec.gov/Archives/edgar/data/6201/000166427224000283/0001664272-24-000283.txt',
 'https://www.sec.gov/Archives/edgar/data/6201/000166427224000282/0001664272-24-000282.txt',
 'https://www.sec.gov/Archives/edgar/data/6201/000166427224000281/0001664272-24-000281.txt',
 'https://www.sec.gov/Archives/edgar/data/6201/000166427224000280/0001664272-24-000280.txt',
 'https://www.sec.gov/Archives/edgar/data/6201/000166427224000279/0001664272-24-000279.txt',
 'https://www.sec.gov/Archives/edgar/data/6201/00016642722400

In [73]:
import re
import requests
import time

# Assuming the URL format ends with the CIK followed by ".txt"
cik_pattern = r'.*/(\d+)\.txt$'

headers = {
    'User-Agent': 'MyEDGARScript/1.0 (varghesb@bc.edu)'  # Replace with your email
}

cik_pattern = r'/(\d+)/\d+-\d+-\d+\.txt$'

# Case 1
for url in filingsCIKListTrimmedLinksList:
    match = re.search(cik_pattern, url)
    if match:
        cik = match.group(1)
        # Adjusted file name format
        file_name = f"{cik}_{url.split('/')[-1]}"
    else:
        cik = 'unknown'
        file_name = f"{cik}_unknown.txt"  # Adjust if needed for handling unknown cases
    
    # Make the request with the headers
    r = requests.get(url, headers=headers)
    
    # Define the file path for each file
    file_path = f"C:\\Users\\sidha\\OneDrive\\Sid\\MSF\\Summer\\RA\\Code\\CodeTest\\{file_name}"
    
    # Save the response text to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(r.text)
    
    print(f"Response saved to {file_path}")
    
    # Wait for a few seconds before making another request (adjust the number of seconds as per the rate limit)
    time.sleep(0.1)

Response saved to C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest\000110465924075153_0001104659-24-075153.txt
Response saved to C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest\000180883424000086_0001808834-24-000086.txt
Response saved to C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest\000076462224000060_0000764622-24-000060.txt
Response saved to C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest\000076462224000057_0000764622-24-000057.txt
Response saved to C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest\000110465924070030_0001104659-24-070030.txt
Response saved to C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest\000166427224000283_0001664272-24-000283.txt
Response saved to C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest\000166427224000282_0001664272-24-000282.txt
Response saved to C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest\000166427224000281_0001664272-24-000281.txt
Response saved to C:\Users\sidha\OneDrive\Sid\MSF\Summer

=============================== COW TEXT SEARCH FUNCTION ===========================

In [ ]:
#6/17
# stop using span. There are too many spans. Too much run time

# find the required sentences
# print out stuff between the <span> and </span> tags along with the line number where sentence is seen

#6/19
#leave context. Goal is to find companies that implement law.
#Return line number and pattern sentence enclosed within "."


#6/30
# Take all <span> enclosed material and store in a list
# Use search function on this list
# Get values

--------------------------------  SPAN FUNCTION --------------------------------------------  

In [22]:
# Code works

import re

def extractSpanText(text):
    lines = text.split('\n')
    
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    #fiduciaryPattern = re.compile(r'fiduciary\s+(duty|loyalty)', re.IGNORECASE)
    
    spanPattern = re.compile(r'<span.*?>', re.IGNORECASE)
    endSpanPattern = re.compile(r'</span>', re.IGNORECASE)

    withinSpan = False
    spanText = ""
    matches = []
    lineNumber = 0
    startColumn = 0

    for line in lines:
        lineNumber += 1
        i = 0
        while i < len(line):
            if not withinSpan:
                start = spanPattern.search(line, i)
                if start:
                    withinSpan = True
                    startColumn = start.start() + 1
                    i = start.end()
                else:
                    break
            else:
                end = endSpanPattern.search(line, i)
                if end:
                    spanText += line[i:end.start()]
                    matches.append((lineNumber, startColumn, spanText.strip()))
                    spanText = ""
                    withinSpan = False
                    i = end.end()
                else:
                    spanText += line[i:]
                    break
    return matches

# Specify the file path
filePath = "testCOW2.txt"

# Read the contents of the file
with open(filePath, 'r', encoding='utf-8') as file:
    text = file.read()

# Extract text between span tags
values = extractSpanText(text)
print(values)

# Print the results for the current file
#print(f"Results for {filePath}:\n")
#for lineNumber, startColumn, value in values:
 #   print(f"Line {lineNumber}, Column {startColumn}: {value}\n")


[(1, 1376, '<br/>'), (21, 1, "The company's policy is to waive certain commercial opportunities. sdgdv The corporate opportunity waiver was included in the company's bylaws.Another section discusses business opportunities that directors might disclaim.It is important to renounce any potential conflicts in commercial opportunities.")]


In [16]:
import re

def extractSpanText(text):
    lines = text.split('\n')
    
    # Define the patterns to search for
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    
    # Define the span patterns
    spanPattern = re.compile(r'<span.*?>', re.IGNORECASE)
    endSpanPattern = re.compile(r'</span>', re.IGNORECASE)

    matches = []

    # Combine lines into a single string for easy pattern searching
    combined_text = '\n'.join(lines)

    # Search for both patterns
    all_patterns = re.compile(f'{opportunityPattern.pattern}|{actionPattern.pattern}', re.IGNORECASE)
    
    for match in all_patterns.finditer(combined_text):
        match_start, match_end = match.span()
        
        # Find the last opening <span> before the match
        span_start_match = list(spanPattern.finditer(combined_text[:match_start]))[-1]
        span_start = span_start_match.start()
        
        # Find the first closing </span> after the match
        span_end_match = endSpanPattern.search(combined_text, match_end)
        span_end = span_end_match.end()
        
        # Extract the entire span content
        span_content = combined_text[span_start:span_end]
        
        # Remove <span> tags and newline characters from span content
        span_text = re.sub(r'<.*?>', '', span_content).replace('\n', ' ')
        
        # Calculate the line number for the starting span
        start_line_number = combined_text.count('\n', 0, span_start) + 1
        
        # Add the result to matches
        matches.append((start_line_number, span_text.strip()))
    
    return matches

# Specify the file path
filePath = "testCOW3.txt"

# Read the contents of the file
with open(filePath, 'r', encoding='utf-8') as file:
    text = file.read()

# Extract text between span tags
values = extractSpanText(text)

# Print the results line by line
for lineNumber, value in values:
    print(f"Line {lineNumber}: {value}")


Line 1: Our business has been materially and adversely affected by the outbreak of the Coronavirus or COVID-19. COVID-19, which has been declared by the World Health Organization to be a &#8220;pandemic,&#8221; has spread to many countries, including the United States, and is impacting domestic and worldwide economic activity. Since being declared a &#8220;pandemic&#8221;, COVID-19 interfered with our ability to meet with certain customers during 2020 and continued into the first half of 2021. In addition, the COVID-19 outbreak has affected the supply chain for many types of products and materials, particularly those being manufactured in China and other countries where the outbreak has resulted in significant disruptions to ongoing business activities.  The corporate opportunity waiver was included in the company's bylaws. Beginning in the second quarter of 2021 and continuing into the first quarter of 2022, we experienced a material disruption in our supply chain as it relates to the

------------------------------------  SEARCH FUNCTION -------------------------------------  

------------------------------------  LINE NUMBER BASED SEARCH -------------------------------------  

In [6]:
# Code Works!



import re
import os

def search(text):
    lines = text.split('\n')
    
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    #fiduciaryPattern = re.compile(r'fiduciary\s+(duty|loyalty)', re.IGNORECASE)
    
    matches = []
       
    for lineNumber in range(len(lines)):
        context = lines[lineNumber:lineNumber+1]
        contextText = " ".join(context)
        
        if (opportunityPattern.search(contextText) and actionPattern.search(contextText)):
            matches.append((lineNumber + 1, lines[lineNumber]))# line number and value
    
    return matches

# test successful
#uncomment below lines for testing
file_path = r"C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\704532_3.1.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

values = search(text)

# Print the results for the current file
print(f"Results for {file_path}:")
for lineNumber, value in values:
    print(f"Line {lineNumber}: {value}")


Results for C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\704532_3.1.txt:


----------------------------------SPAN + SEARCH ------------------------------------------

In [12]:
import re
import os

def search(text):
    lines = text.split('\n')
    
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    
    matches = []
    
    for lineNumber in range(len(lines)):
        context = lines[lineNumber:lineNumber+1]
        contextText = " ".join(context)
        
        if (opportunityPattern.search(contextText) and actionPattern.search(contextText)):
            start_line = lineNumber
            end_line = lineNumber
            
            # Find the start of the <span> tag
            while start_line >= 0:
                span_start_match = re.search(r'<span\b.*?>', lines[start_line], re.IGNORECASE)
                if span_start_match:
                    break
                start_line -= 1
            
            # Find the end of the </span> tag
            while end_line < len(lines):
                if '</span>' in lines[end_line]:
                    break
                end_line += 1
            
            # Extract the content within the <span> tags
            span_text = ' '.join(lines[start_line:end_line+1])
            span_text = re.sub(r'<span\b.*?>', '', span_text, flags=re.IGNORECASE)
            span_text = re.sub(r'</span>', '', span_text, flags=re.IGNORECASE)
            span_text = span_text.strip()
            
            if (opportunityPattern.search(span_text) and actionPattern.search(span_text)):
                matches.append((start_line + 1, span_text))
    
    return matches

# test successful
#uncomment below lines for testing
#file_path = r"C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\10-K\DE\352991.txt"
file_path = "testCOW3.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

values = search(text)

# Print the results for the current file
print(f"Results for {file_path}:")
for lineNumber, value in values:
    print(f"Line {lineNumber}: {value}\n")


Results for testCOW3.txt:
Line 1: </div><ix:continuation continuedAt="_SignificantAccountingPoliciesTextBlock-c0_cont_57" id="_SignificantAccountingPoliciesTextBlock-c0_cont_56"><ix:continuation continuedAt="_GoingConcernAndCovid19PolicyTextBlock-c0_cont_2" id="_GoingConcernAndCovid19PolicyTextBlock-c0_cont_1"><p style="font: 10pt Times New Roman, Times, Serif; text-indent: 0.5in; margin-top: 0pt; margin-bottom: 0pt; text-align: justify">Our business has been materially and adversely affected by the outbreak of the Coronavirus or COVID-19. COVID-19, which has been declared by the World Health Organization to be a &#8220;pandemic,&#8221; has spread to many countries, including the United States, and is impacting domestic and worldwide economic activity. Since being declared a &#8220;pandemic&#8221;, COVID-19 interfered with our ability to meet with certain customers during 2020 and continued into the first half of 2021. In addition, the COVID-19 outbreak has affected the supply chain fo

--------------------------- SENTENCE BASED SEARCH--------------------------------

In [24]:
import re

def search(text):
    sentences = text.split('.')
    
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    #fiduciaryPattern = re.compile(r'fiduciary\s+(duty|loyalty)', re.IGNORECASE)
    
    matches = []
       
    for sentenceNumber in range(len(sentences)):
        context = sentences[max(0, sentenceNumber):sentenceNumber+1]
        contextText = ". ".join(context)
        
        if (opportunityPattern.search(contextText) and actionPattern.search(contextText)):# or fiduciaryPattern.search(contextText):
            matches.append((sentenceNumber + 1, sentences[sentenceNumber].strip()))
    
    return matches
# U.S.A or D.A
# Test the updated search function
file_path = "testCOW3.0.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

values = search(text)

# Print the results for the current file
print(f"Results for {file_path}:")
for sentenceNumber, value in values:
    print(f"Sentence {sentenceNumber}: {value}")


Results for testCOW3.0.txt:
Sentence 5: The corporate opportunity waiver was included in the company's bylaws


In [34]:
# Code Works!
# lite span, lite context
# Sentence based searching



import re

def search(text):
    # Split the text into sentences using periods.
    sentences = text.split('.')
    
    # Define the regex patterns for opportunity and action keywords.
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    
    matches = []
       
    for sentenceNumber in range(len(sentences)):
        sentence = sentences[sentenceNumber].strip()  # Strip any leading/trailing whitespace
        if sentence:  # Ensure the sentence is not empty
            # Check if both patterns are present in the sentence.
            if opportunityPattern.search(sentence) and actionPattern.search(sentence):
                matches.append((sentenceNumber + 1, sentence))
    
    return matches

# Test the updated search function
file_path = "testCOW3.0.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

values = search(text)

# Print the results for the current file
print(f"Results for {file_path}:")
for sentenceNumber, value in values:
    print(f"Sentence {sentenceNumber}: {value}")


Results for testCOW3.0.txt:
Sentence 4: ssssssssssssssssssmcmcmcmcm
scsiwimwoc The corporate opportunity waiver was included in the company's bylaws
Sentence 6: Another
 section
discusses business opportunities that directors might disclaim
Sentence 8: disclaim corporate business opportunities sds


In [1]:
#Code Works!
# Aggregates all text enclosed in <span>. Separates sentences based on delimiter "."
# Test any file locally


from bs4 import BeautifulSoup
import re

def searchSentencesInConsecutiveSpans(htmlContent):
    soup = BeautifulSoup(htmlContent, 'html.parser')
    
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    
    matches = []
    
    # Find all <span> tags
    spanTags = soup.find_all('span')
    
    # Aggregate text from consecutive <span> tags
    aggregatedText = []
    currentText = []
    
    for span in spanTags:
        spanText = span.get_text(separator=' ', strip=True)
        
        # Check if there is a sentence split
        if currentText and not currentText[-1].endswith('.'):
            # If last element in current text doesn't end with a period, merge with current span text
            currentText.append(spanText)
        else:
            # Otherwise, start a new aggregation
            if currentText:
                aggregatedText.append(' '.join(currentText))
            currentText = [spanText]
    
    # Add the last accumulated span text
    if currentText:
        aggregatedText.append(' '.join(currentText))
    
    # Process aggregated text for sentences
    for text in aggregatedText:
        # Split into sentences based on period
        sentences = text.split('.')
        
        for sentence in sentences:
            sentence = sentence.strip()
            if sentence:  # Ensure the sentence is not empty
                # Check if both patterns are present in the sentence
                if opportunityPattern.search(sentence) and actionPattern.search(sentence):
                    matches.append(sentence)
    
    return matches

# Test the function with your HTML content

filePath = 'testCOW3.0.txt'

with open(filePath, 'r', encoding='utf-8') as file:
    htmlContent = file.read()

# Use the function with the file content
matches = searchSentencesInConsecutiveSpans(htmlContent)

# Print the matches found
print("Matches found:\n")
for match in matches:
    print(match + "\n")


Matches found:

The corporate opportunity waiver was

Another
 section
discusses business opportunities that directors might disclaim



In [4]:

# Same code as above
# Iterated across 10k/DE


import os
from bs4 import BeautifulSoup
import re

def searchSentencesInConsecutiveSpans(htmlContent):
    soup = BeautifulSoup(htmlContent, 'html.parser')
    
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    
    matches = []
    
    # Find all <span> tags
    spanTags = soup.find_all('span')
    
    # Aggregate text from consecutive <span> tags
    aggregatedText = []
    currentText = []
    
    for span in spanTags:
        spanText = span.get_text(separator=' ', strip=True)
        
        # Check if there is a sentence split
        if currentText and not currentText[-1].endswith('.'):
            # If last element in current text doesn't end with a period, merge with current span text
            currentText.append(spanText)
        else:
            # Otherwise, start a new aggregation
            if currentText:
                aggregatedText.append(' '.join(currentText))
            currentText = [spanText]
    
    # Add the last accumulated span text
    if currentText:
        aggregatedText.append(' '.join(currentText))
    
    # Process aggregated text for sentences
    for text in aggregatedText:
        # Split into sentences based on period
        sentences = text.split('.')
        
        for sentence in sentences:
            sentence = sentence.strip()
            if sentence:  # Ensure the sentence is not empty
                # Check if both patterns are present in the sentence
                if opportunityPattern.search(sentence) and actionPattern.search(sentence):
                    matches.append(sentence)
    
    return matches

# Directory containing the files
#directoryPath = r'C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\10-K\DE'
directoryPath = r'C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest'




fileCounter = 0
maxFiles = 5

for filename in os.listdir(directoryPath):
    if fileCounter >= maxFiles:
        break
    if filename.endswith('.txt'):  # Ensure the file is a .txt file
        filePath = os.path.join(directoryPath, filename)
        
        with open(filePath, 'r', encoding='utf-8') as file:
            htmlContent = file.read()
        
        # Use the function with the file content
        matches = searchSentencesInConsecutiveSpans(htmlContent)
        
        # Print the matches found
        if matches:
            print(f"Matches found in file {filename}:\n")
            
            
            for match in matches:
                print(match + "\n")
        else:
            print(f"No matches found in file {filename}:\n ")
        
        fileCounter += 1
# Iterate through all files in the directory
#for filename in os.listdir(directoryPath):
#    if filename.endswith('.txt'):  # Ensure the file is a .txt file
 #       filePath = os.path.join(directoryPath, filename)
        
 #       with open(filePath, 'r', encoding='utf-8') as file:
  #          htmlContent = file.read()
        
        # Use the function with the file content
   #     matches = searchSentencesInConsecutiveSpans(htmlContent)
        
        # Print the matches found
    #    if matches:
     #       print(f"Matches found in file {filename}:\n")
      #      for match in matches:
       #         print(match + "\n")


No matches found in file 0000001750_10K.txt:
 
No matches found in file 0000001750_10Q.txt:
 
No matches found in file 0000001750_S8.txt:
 
No matches found in file 0000001800_Prospectus.txt:
 


In [15]:
# p tags instead of span

import os
from bs4 import BeautifulSoup
import re

def searchSentencesInConsecutiveParagraphs(htmlContent):
    soup = BeautifulSoup(htmlContent, 'html.parser')
    
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    
    matches = []
    
    # Find all <p> tags
    pTags = soup.find_all('p')
    
    # Aggregate text from <p> tags
    aggregatedText = []
    
    for p in pTags:
        pText = p.get_text(separator=' ', strip=True)
        if pText:
            aggregatedText.append(pText)
    
    # Process aggregated text for sentences
    for text in aggregatedText:
        # Split into sentences based on period
        sentences = text.split('.')
        
        for sentence in sentences:
            sentence = sentence.strip()
            if sentence:  # Ensure the sentence is not empty
                # Check if both patterns are present in the sentence
                if opportunityPattern.search(sentence) and actionPattern.search(sentence):
                    matches.append(sentence)
    
    return matches

# Directory containing the files
directoryPath = r'C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\10-K\DE'
#directoryPath = r'C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest'

fileCounter = 0

maxFiles = 500
#maxFiles = 10000

cowCompaniesCIKList = []

for filename in os.listdir(directoryPath):
    if fileCounter >= maxFiles:
        break
    if filename.endswith('.txt'):  # Ensure the file is a .txt file
        filePath = os.path.join(directoryPath, filename)
        
        with open(filePath, 'r', encoding='utf-8') as file:
            htmlContent = file.read()
        
        # Use the function with the file content
        matches = searchSentencesInConsecutiveParagraphs(htmlContent)
        
        # Print the matches found
        if matches:
            print(f"Matches found in file {filename}:\n")
            for match in matches:
                print(match + "\n")
            cowCompaniesCIKList.append{filename}
        else:
            print(f"No matches found in file {filename}.\n")
        
        fileCounter += 1


SyntaxError: invalid syntax (3669327242.py, line 68)

In [ ]:
#Matches:#Matches found in file 1083301.txt, 1172358.txt:

# CIK
# Company name
# Year
# Information
# Where it is located

# Use dict?
# dict

In [5]:

#########################STEP 5
# p tags instead of span
import os
from bs4 import BeautifulSoup
import re

def searchSentencesInConsecutiveParagraphs(htmlContent):
    soup = BeautifulSoup(htmlContent, 'html.parser')
    
    opportunityPattern = re.compile(r'(corporate|commercial|business)\s+\S*opportun\w*', re.IGNORECASE)
    actionPattern = re.compile(r'(waiv\w*|renounc\w*|disclaim\w*)', re.IGNORECASE)
    
    matches = []
    
    
    pTags = soup.find_all('p')
    
   
    aggregatedText = []
    
    for p in pTags:
        pText = p.get_text(separator=' ', strip=True)
        if pText:
            aggregatedText.append(pText)
    
    
    for text in aggregatedText:

        sentences = text.split('.')
        
        for sentence in sentences:
            sentence = sentence.strip()
            if sentence: 
                if opportunityPattern.search(sentence) and actionPattern.search(sentence):
                    matches.append(sentence)
    
    return matches


directoryPath = r'C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\10-K\DE'
#directoryPath = r'C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\CodeTest'

fileCounter = 0
#maxFiles = 150
#maxFiles = 100
#maxFiles = 500
maxFiles = 10000
cowCompaniesCIKDict = {}
cowCompaniesCIKList = []
    
for filename in os.listdir(directoryPath):               # for ALL files within a directory
    if fileCounter >= maxFiles:                         # for x files within a directory
       
        break
    if filename.endswith('.txt'):  
        filePath = os.path.join(directoryPath, filename)
        with open(filePath, 'r', encoding='utf-8') as file:
            htmlContent = file.read()


            matches = searchSentencesInConsecutiveParagraphs(htmlContent)

            if matches:
                print(f"Matches found in file {filename}:\n")
                print("\n".join(matches) + "\n")

                cik = os.path.splitext(filename)[0]
                cowCompaniesCIKDict[cik] = matches
                cowCompaniesCIKList.append(cik)
                #for match in matches:
                 #   

                #cowCompaniesCIKList.append({filename})

            else:
                print(f"No matches found in file {filename}.\n")

        fileCounter += 1
#print(cowCompaniesCIKList)


No matches found in file 1000683.txt.

No matches found in file 1000694.txt.

No matches found in file 1001171.txt.

No matches found in file 1001250.txt.

No matches found in file 1001601.txt.

No matches found in file 1001614.txt.

No matches found in file 1001838.txt.

No matches found in file 1001907.txt.

No matches found in file 1002135.txt.

No matches found in file 1002517.txt.

No matches found in file 1002590.txt.

No matches found in file 1003201.txt.

No matches found in file 1004702.txt.

No matches found in file 10048.txt.

No matches found in file 100493.txt.

No matches found in file 1004989.txt.

No matches found in file 1005011.txt.

No matches found in file 1005210.txt.

No matches found in file 1005731.txt.

No matches found in file 100591.txt.

No matches found in file 1006028.txt.

No matches found in file 1006281.txt.

No matches found in file 1007587.txt.

No matches found in file 1008586.txt.

No matches found in file 1008654.txt.

No matches found in file 1009

No matches found in file 1124796.txt.

No matches found in file 1124804.txt.

No matches found in file 1124941.txt.

No matches found in file 1125345.txt.

No matches found in file 1127703.txt.

No matches found in file 1128353.txt.

No matches found in file 1128361.txt.

No matches found in file 1129155.txt.

No matches found in file 1130598.txt.

No matches found in file 1130713.txt.

No matches found in file 1130758.txt.

No matches found in file 1132105.txt.

No matches found in file 1133311.txt.

No matches found in file 1136174.txt.

No matches found in file 1136294.txt.

No matches found in file 1136352.txt.

No matches found in file 1136869.txt.

No matches found in file 1138118.txt.

No matches found in file 1138724.txt.

No matches found in file 1140215.txt.

No matches found in file 1140859.txt.

No matches found in file 1141103.txt.

No matches found in file 1142417.txt.

No matches found in file 1142596.txt.

No matches found in file 1142750.txt.

No matches found in file 

No matches found in file 1364885.txt.

No matches found in file 1365038.txt.

No matches found in file 1366246.txt.

No matches found in file 1366868.txt.

No matches found in file 1367083.txt.

No matches found in file 1367306.txt.

No matches found in file 1367644.txt.

No matches found in file 1368265.txt.

No matches found in file 1368275.txt.

No matches found in file 1368365.txt.

No matches found in file 1368622.txt.

No matches found in file 1368761.txt.

No matches found in file 1369568.txt.

No matches found in file 1370053.txt.

No matches found in file 1370637.txt.

No matches found in file 1370880.txt.

No matches found in file 1371571.txt.

No matches found in file 1372299.txt.

No matches found in file 1372514.txt.

No matches found in file 1372612.txt.

No matches found in file 1374535.txt.

No matches found in file 1374881.txt.

No matches found in file 1377121.txt.

No matches found in file 1377178.txt.

No matches found in file 1377789.txt.

No matches found in file 

No matches found in file 1496671.txt.

No matches found in file 1497253.txt.

No matches found in file 1497504.txt.

No matches found in file 1498067.txt.

No matches found in file 1498232.txt.

Matches found in file 1498233.txt:

Our amended and restated
certificate of incorporation provides that we renounce our interest in any corporate opportunity offered to any director or officer unless
such opportunity is expressly offered to such person solely in his or her capacity as a director or officer of our company and such opportunity
is one we are legally and contractually permitted to undertake and would otherwise be reasonable for us to pursue, and to the extent the
director or officer is permitted to refer that opportunity to us without violating another legal obligation
Our amended and restated certificate
of incorporation provides that we renounce our interest in any corporate opportunity offered to any director or officer unless such opportunity
is expressly offered to such person

No matches found in file 1579214.txt.

No matches found in file 1580490.txt.

No matches found in file 1580670.txt.

No matches found in file 1580808.txt.

No matches found in file 1581280.txt.

No matches found in file 1581990.txt.

No matches found in file 1582961.txt.

No matches found in file 1582982.txt.

No matches found in file 1583708.txt.

No matches found in file 1583771.txt.

No matches found in file 1584509.txt.

No matches found in file 15847.txt.

No matches found in file 1584754.txt.

No matches found in file 1585608.txt.

No matches found in file 1585673.txt.

No matches found in file 1587221.txt.

No matches found in file 1587246.txt.

No matches found in file 1588272.txt.

No matches found in file 1588489.txt.

No matches found in file 1589149.txt.

No matches found in file 1590383.txt.

No matches found in file 1590895.txt.

No matches found in file 1590976.txt.

No matches found in file 1591588.txt.

No matches found in file 1591698.txt.

No matches found in file 15

...

Matches found in file 1863990.txt:

Our amended and restated certificate of incorporation provides that, subject to fiduciary duties under Delaware law, we renounce our interest in any corporate opportunity offered to any director or officer unless such opportunity is expressly offered to such person solely in his or her capacity as a director or officer of our company and such opportunity is one we are legally and contractually permitted to undertake and would otherwise be reasonable for us to pursue
Notwithstanding the foregoing, our amended and restated certificate of incorporation provides that, subject to fiduciary duties under Delaware law, we renounce our interest in any corporate opportunity offered to any director or officer unless such opportunity is expressly offered to such person solely in his or her capacity as a director or officer of our company and such opportunity is one we are legally and contractually permitted to undertake and would otherwise be reasonable for us t

No matches found in file 1876588.txt.

No matches found in file 1876620.txt.

No matches found in file 1878897.txt.

No matches found in file 1879726.txt.

Matches found in file 1880151.txt:

Our amended and restated certificate of incorporation
provides that we renounce our interest in any corporate opportunity offered to any director or officer unless such opportunity is expressly
offered to such person solely in his or her capacity as a director or officer of our company, and such opportunity is one we are legally
and contractually permitted to undertake and would otherwise be reasonable for us to pursue, and to the extent the director or officer
is permitted to refer that opportunity to us without violating another legal obligation

No matches found in file 1880613.txt.

No matches found in file 1884164.txt.

No matches found in file 1884697.txt.

No matches found in file 1885998.txt.

No matches found in file 1888654.txt.

No matches found in file 1888734.txt.

No matches found in

C:\Users\sidha\Anaconda2\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


No matches found in file 48465.txt.

No matches found in file 50725.txt.

No matches found in file 50863.txt.

No matches found in file 52428.txt.

No matches found in file 52988.txt.

No matches found in file 54480.txt.

No matches found in file 5513.txt.

No matches found in file 55529.txt.

No matches found in file 56679.txt.

No matches found in file 56701.txt.

No matches found in file 56868.txt.

No matches found in file 57725.txt.

No matches found in file 58361.txt.

No matches found in file 5981.txt.

No matches found in file 60714.txt.

No matches found in file 63276.txt.

No matches found in file 64472.txt.

No matches found in file 65270.txt.

No matches found in file 65312.txt.

No matches found in file 6769.txt.

No matches found in file 69422.txt.

No matches found in file 69633.txt.

No matches found in file 69733.txt.

No matches found in file 69891.txt.

No matches found in file 701288.txt.

No matches found in file 701869.txt.

No matches found in file 703351.txt.

N

No matches found in file 876523.txt.

No matches found in file 877890.txt.

No matches found in file 87802.txt.

No matches found in file 878526.txt.

No matches found in file 878726.txt.

No matches found in file 878932.txt.

No matches found in file 879407.txt.

No matches found in file 879526.txt.

No matches found in file 879911.txt.

No matches found in file 880117.txt.

No matches found in file 880266.txt.

No matches found in file 880432.txt.

No matches found in file 880631.txt.

No matches found in file 880807.txt.

No matches found in file 88205.txt.

No matches found in file 882095.txt.

No matches found in file 882184.txt.

No matches found in file 882796.txt.

No matches found in file 882800.txt.

No matches found in file 883107.txt.

No matches found in file 883237.txt.

No matches found in file 883241.txt.

No matches found in file 883569.txt.

No matches found in file 883902.txt.

No matches found in file 883975.txt.

No matches found in file 883984.txt.

No matches fou

In [6]:
#########################STEP 6
cowCompaniesCIKDict
#cowCompaniesCIKList

{'1083301': ['In addition, the amended and restated certificate of incorporation of TeraWulf provides that TeraWulf renounces any interest or expectancy with respect to, among others, business opportunities relating to the bitcoin mining business in Hardin, Montana, in connection with building and operation by affiliates of Beowulf Energy LLC of a data center for Marathon Digital Holdings Inc'],
 '1172358': ['we waive any claim that any business opportunity pursued by our General Partner, the GP Parties or their affiliates and the manager designees constitutes a corporate opportunity that should have been presented to us'],
 '1498233': ['Our amended and restated\ncertificate of incorporation provides that we renounce our interest in any corporate opportunity offered to any director or officer unless\nsuch opportunity is expressly offered to such person solely in his or her capacity as a director or officer of our company and such opportunity\nis one we are legally and contractually per

In [7]:
#########################STEP 7
# Code Works!!
# Prints matched companies' parameters using earlier companyParams dict

cowCompanyParams = {}
#cowCompaniesCIKList
for cik in companyParams:
    if cik in cowCompaniesCIKDict:
        cowCompanyParams[cik] = {
            **companyParams[cik],  # Add all values from companyParams
            "matches": cowCompaniesCIKDict[cik]  # Add matches from cowCompaniesCIKDict
        }
       
        #print(companyParams[i] , ",", cowCompaniesCIKDict[i])

    

#len(cowCompaniesCIKList)

In [8]:
cowCompanyParams

{'1869974': {'cik': '1869974',
  'companyName': 'Aesther Healthcare Acquisition Corp.',
  'periodOfReport': '2021-12-31',
  'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1869974/000149315222002707/0001493152-22-002707.txt',
  'matches': ['Our amended and restated certificate of incorporation provides that we renounce our interest in any corporate opportunity offered to\nany director or officer unless such opportunity is expressly offered to such person solely in his or her capacity as a director or officer\nof our company and such opportunity is one we are legally and contractually permitted to undertake and would otherwise be reasonable\nfor us to pursue, and to the extent the director or officer is permitted to refer that opportunity to us without violating another legal\nobligation',
   'Our amended and restated certificate of incorporation provides that we renounce our interest in any corporate opportunity offered to\nany director or officer unless such opportunity is expre

In [9]:
len(cowCompanyParams)

151

In [10]:
cowCompanyParams.keys()

dict_keys(['1869974', '1814215', '1817640', '1824013', '1821075', '1743759', '1560385', '1847351', '1172358', '1821788', '1704596', '1817153', '1829328', '1823086', '1805795', '1817868', '1855168', '1821812', '1819498', '1829999', '1826729', '1858912', '1083301', '1615063', '1842883', '1870471', '910612', '1838814', '1865191', '1844505', '1838219', '1848795', '1815737', '1858503', '1853651', '1845618', '1839530', '1839519', '1863099', '1825249', '1831270', '1828608', '1820160', '1863719', '1817565', '1865200', '1824677', '1852633', '1841661', '1600132', '1837105', '1842046', '1849056', '1828248', '1841125', '1865120', '1880151', '1859007', '1822835', '1839998', '1828325', '1753539', '1817786', '1833235', '1832250', '1853920', '1820875', '1802749', '1816017', '1867757', '1828096', '1863990', '1824920', '1833214', '1836274', '1821850', '1824734', '1818382', '1817232', '1818644', '1837607', '1834342', '1823587', '1830210', '1822219', '1855302', '1819496', '1823826', '1824993', '1827980', 